# Answering Business Questions Using SQL #

## Introduction ##

In this project we will be working with a database called Chinook. This database contains information about a fictional digital music shop kind of like a mini-iTunes store.
The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

Here's a schema diagram for the Chinook database:

<img src="https://tinypic.host/images/2023/12/02/chinook_esquema.jpeg" alt="chinook_esquema.jpeg" border="0">

We will use this database and the sqlite3 module in order to explore and analyze four fictional business questions and propositions.

## Set up and Overview of the Data##

We first have to connect our Jupyter Notebook to our database file. Then, we will be able to do the first scanning of the Chinook Database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [16]:
%%sql
SELECT * 
FROM customer
LIMIT 3;


Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


## Question 1: Selecting Albums to Purchase ##

We have to select the first three albums, from a list of four albums, that will be added to the Chinook record store from a new record label. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:
* Regal: Hip-Hop
* Red Tone: Punk
* Meteor and the Girls: Pop
* Slim Jim Bites: Blues

We will find out which genres sell the most tracks in the USA, and make a recommendation for the three artists whose albums we should purchase for the store.

In [31]:
%%sql 
       WITH 
        usa AS
        (SELECT cus.customer_id, cus.country FROM customer AS cus
        WHERE country='USA'
       ),
        track_invoice_genre AS
        (SELECT i.invoice_id, i.customer_id, il.track_id,t.name,t.genre_id, g.name AS genre
         FROM invoice AS i
         INNER JOIN invoice_line AS il ON i.invoice_id=il.invoice_id
         INNER JOIN track AS t ON il.track_id=t.track_id
         INNER JOIN genre AS g ON t.genre_id=g.genre_id
        ),
         usa_track_genre AS
        (SELECT u.customer_id,u.country,tig.customer_id,tig.invoice_id,tig.track_id,tig.name,tig.genre_id,tig.genre
         FROM usa AS u
         INNER JOIN track_invoice_genre AS tig ON u.customer_id=tig.customer_id
         )
    

SELECT utg.genre, COUNT(utg.track_id) AS number_tracks,
       CAST(COUNT(utg.track_id) AS FLOAT)/(SELECT COUNT(utg.track_id) FROM usa_track_genre AS utg) AS percentage
FROM usa_track_genre AS utg
GROUP BY utg.genre
ORDER BY percentage DESC;






Done.


genre,number_tracks,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


The most popular genre corresponds to Rock, followed by Alternative & Punk and Metal. 
No doubt, we recommend selecting the Red Tone album (punk genre) to include it in the store. The Hip Hop genre is the least popular genre included on the list of four albums, therefore we recomend excluding the corresponding album (Regal: Hip Hop) from the selection and keep:
* Red Tone: Punk
* Slim Jim Bites: Blues
* Meteor and the Girls: Pop

## Question 2: Analyzing Employee Sales Performance ##

We will analyze the purchases of customers assigned to each employee to determine if any sales support agent is performing better or worse than the others. 

Our assessment will consider total sales amounts, employee birthdates, and tenure with the company.



In [32]:
%%sql
WITH customer_invoice AS
     (SELECT c.support_rep_id, SUM(i.total) AS total
      FROM customer AS c
      INNER JOIN invoice AS i ON i.customer_id=c.customer_id
      GROUP BY support_rep_id
      ORDER BY support_rep_id)
SELECT e.first_name || " " || e.last_name AS employee_name, e.hire_date AS hire_date, e.birthdate AS birthdate, ci.total AS total 
FROM employee AS e
INNER JOIN customer_invoice AS ci ON ci.support_rep_id=e.employee_id;


Done.


employee_name,hire_date,birthdate,total
Jane Peacock,2017-04-01 00:00:00,1973-08-29 00:00:00,1731.510000000004
Margaret Park,2017-05-03 00:00:00,1947-09-19 00:00:00,1584.0000000000032
Steve Johnson,2017-10-17 00:00:00,1965-03-03 00:00:00,1393.9200000000028


There is a 19.5% variation in sales between Jane Peacock, who ranks highest, and Steven Johnson, who ranks lowest. However, this variation can be attributed to differences in their respective hiring dates, spanning a range of about six months. Moreover, Jane Peacock, the agent with the highest amout of sales, is the youngest, which might reflect an ability to connect to a younger customer market.

## Question 3: Analyzing Sales by Country ##

For this question we will analyze the sales data for customers from each country. 

We will calculate for each country:

* Total number of customers
* Total value of sales
* Average value of sales per customer
* Average order value

In [6]:
%%sql
WITH customer_invoice AS
    (SELECT c.customer_id AS customer_id, c.country AS country, i.invoice_id AS invoice_id, i.total AS total
     FROM customer AS c
     INNER JOIN invoice AS i ON i.customer_id=c.customer_id
     ORDER BY c.country
    ), 
     total_customer_sales AS
    (SELECT COUNT(DISTINCT invoice_id) AS number_invoices, country, COUNT(DISTINCT customer_id) AS number_customers, SUM(total) total_sales
     FROM customer_invoice AS ci
     GROUP BY country
     ORDER BY country
    )
SELECT _country_,number__customers,total__sales, total__sales/number__customers AS average_sales_customer, total__sales/number__invoices AS average_order_value
FROM (SELECT t.*,
       CASE 
           WHEN t.number_customers=1 THEN 1
           ELSE 0
       END AS sort,
       CASE
        WHEN t.number_customers > 1 THEN t.country
        ELSE 'Others'
       END AS _country_,
       CASE 
        WHEN t.number_customers>1 THEN t.total_sales
        ELSE SUM(t.total_sales) 
        END AS total__sales,
       CASE 
        WHEN t.number_customers>1 THEN t.number_customers
        ELSE SUM(t.number_customers) 
        END AS number__customers,
       CASE 
        WHEN t.number_customers>1 THEN t.number_invoices
        ELSE SUM(t.number_invoices) 
        END AS number__invoices
    FROM total_customer_sales AS t
    GROUP BY _country_
      )
    ORDER BY sort ASC, total__sales DESC;

Done.


_country_,number__customers,total__sales,average_sales_customer,average_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Others,15,1094.9399999999998,72.996,7.4485714285714275


Analyzing the data suggests potential opportunities in the following countries: Czech Republic, United Kingdom, and India. 

However, it's important to note that the data volume from these countries is relatively small. Therefore, we should be cautious when deciding where to spend money for new marketing campaigns, as the sample size isn't substantial enough to provide a high level of confidence. A more prudent approach would involve conducting small-scale campaigns in these countries, allowing us to collect and analyze data from new customers to ensure that these trends persist among this customer segment.

## Question 4: Albums vs Individual Tracks ##

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.



In [7]:
%%sql
WITH invoice_track AS (
   SELECT invoice_id, t.track_id, album_id
   FROM invoice_line AS il
   INNER JOIN track AS t ON t.track_id = il.track_id
),
first_track AS (
    SELECT invoice_id, MIN(track_id) AS first_track, album_id AS album_id
    FROM invoice_track AS ita
    GROUP BY invoice_id
),
album_purchases AS (
   SELECT
      it.invoice_id,
      it.album_id,
      CASE
        WHEN (
          SELECT t.track_id
          FROM track t
          WHERE t.album_id = it.album_id
          
          EXCEPT
          
          SELECT il2.track_id
          FROM invoice_line il2
          WHERE il2.invoice_id = it.invoice_id
        ) IS NULL
        AND (
          SELECT il2.track_id
          FROM invoice_line il2
          WHERE il2.invoice_id = it.invoice_id
          
          EXCEPT
          
          SELECT t.track_id
          FROM track t
          WHERE t.album_id = it.album_id
        ) IS NULL
        THEN 'yes'
        ELSE 'no'
      END AS album_purchased
   FROM invoice_track AS it
)
SELECT
  album_purchased,
  COUNT(DISTINCT invoice_id) AS number_of_invoices,
  CAST(COUNT(DISTINCT invoice_id) AS FLOAT) / (
    SELECT COUNT(DISTINCT invoice_id) FROM invoice_track
  ) AS percent
FROM album_purchases
GROUP BY album_purchased;



Done.


album_purchased,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Out of all purchases made, album purchases represent 18.6%. Given this information, I would advise against buying individual tracks from record company albums exclusively, as it could result in a potential loss of approximately 20% of revenue.

## Conclusions ##

In this project, we used the sqlite3 module to examine Chinook's sales data. Our goal was to gain insights into the sales dynamics and employee performance. We addressed strategic business questions within fictional scenarios, such as introducing new artists to the product portfolio and considering a change in purchasing strategy.